(c) 2016 - present. Enplus Advisors, Inc.

This module uses:
* SP500 returns
* Security prices

In [1]:
from datetime import datetime

import dateutil
import numpy as np
import pandas as pd

In [2]:
sp5_df = pd.read_csv('sp500.csv', usecols=['date', 'adj_close'])
sp5_dts = pd.to_datetime(sp5_df.date)

In [3]:
prices = pd.read_csv('prices.csv', parse_dates=['date'])
prices = prices.set_index('date', verify_integrity=True).sort_index()

**Exercise:**

Create a `pandas` Timestamp for January 1st, 1993 4:00 PM.

In [4]:
ts = pd.Timestamp('1993-01-01 00:16:00')

**Exercise:**

Generate a an Index of:
* 500 calendar days starting on January 01, 2010.
* 1 year of US business days starting on January 01, 2010

In [5]:
cd500 = pd.date_range(start='2010-01-01', periods=500, freq='D')

In [6]:
bd2010 = pd.date_range(start='2010-01-01', end='2010-12-31', freq='B')

**Exercise:**

* Create a Series from the `adj_close` column `sp5_df`, using `date` as a `DatetimeIndex`
* Sort the index


In [7]:
sp5 = pd.Series(sp5_df.adj_close.values, index=sp5_dts, name='adj_close').sort_index()

**Exercise:**

Write 3 different ways to select January 2, 1995 from the `sp5` series.

In [8]:
d1a = sp5['19950103']
d1b = sp5['1995-01-03']
d1c = sp5[datetime(1995, 1, 3)]

**Exercise:**

Select from the `sp5` DataFrame all observations for:
* March, 1995
* Year of 1995

In [20]:
mar_95 = sp5['1995-03']

date
1995-03-01    485.649994
1995-03-02    485.130005
1995-03-03    485.420013
1995-03-06    485.630005
1995-03-07    482.119995
1995-03-08    483.140015
1995-03-09    483.160004
1995-03-10    489.570007
1995-03-13    490.049988
1995-03-14    492.890015
1995-03-15    491.880005
1995-03-16    495.410004
1995-03-17    495.519989
1995-03-20    496.140015
1995-03-21    495.070007
1995-03-22    495.670013
1995-03-23    495.950012
1995-03-24    500.970001
1995-03-27    503.200012
1995-03-28    503.899994
1995-03-29    503.119995
1995-03-30    502.220001
1995-03-31    500.709991
Name: adj_close, dtype: float64

In [10]:
y_95 = sp5['1995']

**Exercise**

For the SP500:
* Calculate 1-day total returns
* Calculate the daily cumulative total return 
* Calculate weekly, monthly, yearly returns.

In [11]:
sp5_rtn = sp5.pct_change().rename('rtn')

In [12]:
sp5_crtn = (1 + sp5_rtn).cumprod().rename('crtn')
sp5_crtn.iat[0] = 1

**Exercise:**

* Calculate 1-day total returns.
* Calculate a cumulative return index for each security.
* Calculate weekly, monthly, yearly returns.

In [13]:
# 1-day total return
rtns = prices.pct_change()

In [14]:
# Cumulative returns
crtns = (1. + rtns).cumprod()
crtns.iloc[0, :] = 1

In [15]:
# Weekly, Monthly, Yearly returns
rtn_wk = crtns.resample('W').last()
rtn_mnth = crtns.resample('M').last()

**Exercise:**

For all securities in prices:

* Calculate a 1-month and 12-month momentum signal.
* Calculate a momentum signal of 12-month returns minus 1-month returns.

In [16]:
# 1 and 12 month momentum
rtn_m1 = crtns - crtns.shift(21)
rtn_m12 = crtns - crtns.shift(252)
mom_12m1 = (rtn_m12 - rtn_m1).dropna()

**Exercise:**

* Calculate the same momentum signal using `rolling`

In [17]:
# Lookback of 252 days, most recent minus day 21
mom_12m1_v2 = crtns.rolling(253).apply(lambda x: x[-22] - x[0]).dropna()

In [18]:
x, y = mom_12m1.align(mom_12m1_v2, join='left')
idx = ((x - y).abs() < 1e-8) & (x.isnull() == y.isnull())
assert idx.all().all()

In [19]:
rng = pd.date_range('20110101', periods=72, freq='H')
ts = pd.Series(np.random.randn(len(rng)), index=rng)